# 1. Import Libraries

In [1]:
import json
import pandas as pd
import numpy as np
import geopandas as gpd

> Note: The 'geopandas' module allows us to work with geospatial data in python easier.

# 2. Preparing Dataset

## 2.1 Yelp Open Dataset

Yelp Open Dataset is an open dataset released by Yelp for learning purposes. It consists of millions of user reviews, businesses attributes and over 200,000 pictures from multiple metropolitan areas
- Size: 2.66 GB JSON 
- Number of Records: 5,200,000 reviews, 174,000 business attributes, 200,000 pictures etc.
     > We only used Business JSON fiel to conduct an analysis. An example of the Business JSON file:
       [{'business_id': 'Apn5Q_b6Nz61Tq4XzPdf9A',
         'name': 'Minhas Micro Brewery',
         'neighborhood': '',
         'address': '1314 44 Avenue NE',
         'city': 'Calgary',
         'state': 'AB',
         'postal_code': 'T2E 6L6',
         'latitude': 51.0918130155,
         'longitude': -114.031674872,
         'stars': 4.0,
         'review_count': 24,
         'is_open': 1,
         'attributes': {'BikeParking': 'False',
          'BusinessAcceptsCreditCards': 'True',
          'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}",
          'GoodForKids': 'True',
          'HasTV': 'True',
          'NoiseLevel': 'average',
          'OutdoorSeating': 'False',
          'RestaurantsAttire': 'casual',
          'RestaurantsDelivery': 'False',
          'RestaurantsGoodForGroups': 'True',
          'RestaurantsPriceRange2': '2',
          'RestaurantsReservations': 'True',
          'RestaurantsTakeOut': 'True'},
         'categories': 'Tours, Breweries, Pizza, Restaurants, Food, Hotels & Travel',
         'hours': {'Monday': '8:30-17:0',
          'Tuesday': '11:0-21:0',
          'Wednesday': '11:0-21:0',
          'Thursday': '11:0-21:0',
          'Friday': '11:0-21:0',
          'Saturday': '11:0-21:0'}},
           ...]

## 2.2 Data Cleaning 

### 2.2.1 Read JSON file

In [2]:
data = []
with open('yelp_academic_dataset_business.json') as f:
    for line in f:
        data.append(json.loads(line))

### 2.2.2 Convert the JSON file to DataFrame and drop the rows if the `attribute` is NAN

In [3]:
df = pd.DataFrame(data)
df.dropna(subset=['attributes'],inplace=True)

### 2.2.3 Insert the `RestaurantsPriceRange2` to the DataFrame ( Yelp $ Signs)

In [4]:
df_attribute = pd.DataFrame([i if i != None else {'RestaurantsPriceRange2':np.NAN} for i in df['attributes']])

In [5]:
df['dollar_sign'] = df_attribute['RestaurantsPriceRange2']

### 2.2.4 Select only important columns

In [6]:
df = df[['business_id', 'categories','latitude','longitude','dollar_sign','city','state']]

In [7]:
df.head()

,business_id,categories,latitude,longitude,dollar_sign,city,state
0,Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",51.091813,-114.031675,2,Calgary,AB
1,AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",35.960734,-114.939821,2,Henderson,NV
2,O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",45.540503,-73.599300,2,Montréal,QC
4,8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",51.035591,-114.027366,1,Calgary,AB
5,45bWSZtniwPRiqlivpS8Og,"Coffee & Tea, Food",33.671375,-112.030017,1,Phoenix,AZ


### 2.2.5 Identify the more accurate zip code based on `latitude` and `longitude`

The cartographic boundary files are simplified representations of selected geographic areas from the Census Bureau’s MAF/TIGER geographic database. 
Source: https://www.census.gov/geo/maps-data/data/cbf/cbf_zcta.html

<font color='red'>Daniel, could you please describe how you generate the 'yelp_geo_zips.csv'?<font>

In [8]:
zip_code = pd.read_csv('https://raw.githubusercontent.com/jlian014/yelp/master/yelp_geo_zips.csv')

In [9]:
zip_code.head()

,business_id,zip
0,Apn5Q_b6Nz61Tq4XzPdf9A,not_in_census_zip
1,AjEbIBw6ZFfln7ePHha9PA,89005
2,O8S5hYJ1SMc8fA4QBtVujA,not_in_census_zip
3,45bWSZtniwPRiqlivpS8Og,85050
4,9A2quhZLyWk0akUetBd8hQ,not_in_census_zip


Finally, we inserted the zip code to the dataframe.

In [10]:
df = df.merge(zip_code,on='business_id',how='inner')

### 2.2.6 Missing values

In [11]:
df.isnull().sum()

business_id        0
categories        31
latitude           3
longitude          2
dollar_sign    46089
city               0
state              0
zip                0
dtype: int64

In [12]:
df['categories'] = df['categories'].fillna('None')

In [13]:
df.dropna(inplace=True)

In [14]:
df.isnull().sum()

business_id    0
categories     0
latitude       0
longitude      0
dollar_sign    0
city           0
state          0
zip            0
dtype: int64

### 2.2.7 Select only 5 digit US postal_code 

In [15]:
df = df[df.zip.str.match("\d{5}")]

### 2.2.8 Group by zipcode

In [16]:
df_post = df.set_index('zip')
pivoted = pd.pivot_table(df_post, index='zip', columns='dollar_sign', values='business_id', aggfunc="count",fill_value=0)
flattened = pd.DataFrame(pivoted.to_records())
df_post = flattened.rename(columns={'1':'yelp_dollar_1','2':'yelp_dollar_2','3':'yelp_dollar_3','4':'yelp_dollar_4'})
df_post['Total_Business'] = df_post['yelp_dollar_1'] + df_post['yelp_dollar_2'] +  df_post['yelp_dollar_3'] + df_post['yelp_dollar_4']

### 2.2.9 Drop the states that have small amount of  zipcode information

In [17]:
states_to_drop = ['NY','TX','CA','IN','VA','KY','CO','OR','TAM','ON']
df = df[~df['state'].isin(states_to_drop)]

In [18]:
df['state'].unique()

array(['NV', 'AZ', 'IL', 'OH', 'WI', 'PA', 'NC', 'SC'], dtype=object)

### 2.2.6 Import IRS dataset

The [IRS 2016 Individual Income Tax Statistics](https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2016-zip-code-data-soi) is our variables to identify the zipcode affluent. The data shows that population that was filed and processed by the IRS during the 2016 calendar year. The money amounts are reported are divided in 6 numerical categories.

                1 = $1 under $25,000
                2 = $25,000 under $50,000
                3 = $50,000 under $75,000
                4 = $75,000 under $100,000
                5 = $100,000 under $200,000
                6 = $200,000 or more

In [ ]:
IRS = pd.read_csv('./IRS_zipcodes_clean.csv')

In [ ]:
IRS.head()

,STATE,zipcode,1,2,3,4,5,6
0,AL,35004,1510.0,1410.0,950.0,650.0,630.0,60.0
1,AL,35005,1310.0,960.0,450.0,200.0,180.0,0.0
2,AL,35006,430.0,330.0,190.0,120.0,130.0,0.0
3,AL,35007,4070.0,2650.0,1820.0,1340.0,1820.0,240.0
4,AL,35010,3620.0,2070.0,890.0,530.0,620.0,180.0


In [ ]:
IRS = IRS.rename(columns={'zipcode':'zip',
                          '1':'income_level_1',
                          '2':'income_level_2',
                          '3':'income_level_3',
                          '4':'income_level_4',
                          '5':'income_level_5',
                          '6':'income_level_6'})

In [ ]:
df_post.dtypes

zip               object
yelp_dollar_1      int64
yelp_dollar_2      int64
yelp_dollar_3      int64
yelp_dollar_4      int64
Total_Business     int64
dtype: object

In [ ]:
df_post.zip = df_post.zip.astype('int64')

In [ ]:
df_post = df_post.merge(IRS,on='zip',how='inner')

Merge the data to the yelp dataset.

### 2.2.7 Target variable `affluence` based on IRS income data

Assign the target variable classes based on the median income level.


In [ ]:
irs_freq_count = np.array(df_post.loc[:, 'income_level_1':'income_level_6'])
cumm = np.cumsum(irs_freq_count, axis=1) 
indices = [np.searchsorted(row, row[-1]/2.0) for row in cumm]
df_post['target'] = [i+1 for i in indices]

In [ ]:
df_post.head()

### 2.2.8 Export the dataset for further use

In [ ]:
df_post.to_csv('yelp_dataset_final.csv')


**The data dictionary of the resulted dataframe:**


|Variables| dtype| description |
|-------|----|-----|
|zip|int64|zipcode|
|yelp_dollar_1|int|the number of businesses that have 1 dollar signs in this zipcode area|
|yelp_dollar_2|int|the number of businesses that have 2 dollar signs in this zipcode area|
|yelp_dollar_3|int|the number of businesses that have 3 dollar signs in this zipcode area|
|yelp_dollar_4|int|the number of businesses that have 4 dollar signs in this zipcode area|
|Total_Business|int|the total number of businesses in this zipcode area|
|STATE|object|States|
|income_level_1|float|the number of people whose income level lie in level_1 category(statistics from IRS)|
|income_level_2|float|the number of people whose income level lie in level_2 category(statistics from IRS)|
|income_level_3|float|the number of people whose income level lie in level_3 category(statistics from IRS)|
|income_level_4|float|the number of people whose income level lie in level_4 category(statistics from IRS)|
|income_level_5|float|the number of people whose income level lie in level_5 category(statistics from IRS)|
|income_level_6|float|the number of people whose income level lie in level_6 category(statistics from IRS)|
|target|int|the target variable shows the neighborhood affluence